<a href="https://colab.research.google.com/github/algvr/ami-detection/blob/main/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import PIL
print('Hello!')